In [1]:
import torch
from torch import nn
from torchinfo import summary
from torchvision import models
from torchvision import transforms
from torchmetrics import Accuracy
from pathlib import Path
import os

import matplotlib.pyplot as plt

from going_modular.going_modular import data_setup, engine

d:\Work\Coding\Machine-Learning\PyTorch\going_modular\going_modular\engine.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Set seeds

In [2]:
def set_seeds(SEED):

    torch.manual_seed(SEED)

    torch.cuda.manual_seed(SEED)

## Device agnositc code

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Set accuracy metrics

In [4]:
train_accuracy_fn = Accuracy(task='multiclass', num_classes=3).to(device)
test_accuracy_fn = Accuracy(task='multiclass', num_classes=3).to(device)

## Let's load model parameters

In [5]:
weights = models.EfficientNet_B0_Weights.DEFAULT
weights

EfficientNet_B0_Weights.IMAGENET1K_V1

## Create the transform functions

In [6]:
## Auto transform
auto_transform = weights.transforms()


# Manual tranform

normalize = transforms.Normalize(
     mean=[0.485, 0.456, 0.406],
     std=[0.229, 0.224, 0.225]
)

manual_transform = transforms.Compose([

    transforms.Resize((244, 244)),
    transforms.ToTensor(),
    normalize
])

auto_transform, manual_transform

(ImageClassification(
     crop_size=[224]
     resize_size=[256]
     mean=[0.485, 0.456, 0.406]
     std=[0.229, 0.224, 0.225]
     interpolation=InterpolationMode.BICUBIC
 ),
 Compose(
     Resize(size=(244, 244), interpolation=bilinear, max_size=None, antialias=True)
     ToTensor()
     Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
 ))

## Get number of cores

In [7]:
os.cpu_count()

12

In [8]:
dataset_path = Path('./datasets/pizza_steak_sushi')
train_dir = dataset_path / 'train'
test_dir = dataset_path / 'test'

## creat_dataloaders parameters

BATCH_SIZE = 32
NUM_WORKERS = 2

train_dataloader, test_dataloader, classes = data_setup.create_dataloaders(train_dir=train_dir,
                              test_dir=test_dir,
                              transform=auto_transform,
                              batch_size=BATCH_SIZE,
                              num_workers=NUM_WORKERS)

train_dataloader, test_dataloader, classes

(<torch.utils.data.dataloader.DataLoader at 0x1cf35bcf310>,
 ['pizza', 'steak', 'sushi'])

## Download the pertrained model

In [9]:
model = models.efficientnet_b0(weights=weights)
summary(model,
        input_size=[32, 3, 244, 244],
        col_names=['num_params', 'input_size', 'output_size', 'trainable'],
        row_settings=['var_names'])

Layer (type (var_name))                                      Param #                   Input Shape               Output Shape              Trainable
EfficientNet (EfficientNet)                                  --                        [32, 3, 244, 244]         [32, 1000]                True
├─Sequential (features)                                      --                        [32, 3, 244, 244]         [32, 1280, 8, 8]          True
│    └─Conv2dNormActivation (0)                              --                        [32, 3, 244, 244]         [32, 32, 122, 122]        True
│    │    └─Conv2d (0)                                       864                       [32, 3, 244, 244]         [32, 32, 122, 122]        True
│    │    └─BatchNorm2d (1)                                  64                        [32, 32, 122, 122]        [32, 32, 122, 122]        True
│    │    └─SiLU (2)                                         --                        [32, 32, 122, 122]        [32, 32, 122, 122]

## Freeze the features

In [10]:
for params in model.features.parameters():

    params.requires_grad = False

## Recreate the classfier

In [11]:
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=len(classes), bias=True)
)

In [12]:
summary(model,
        input_size=[32, 3, 244, 244],
        col_names=['num_params', 'input_size', 'output_size', 'trainable'],
        row_settings=['var_names'])

Layer (type (var_name))                                      Param #                   Input Shape               Output Shape              Trainable
EfficientNet (EfficientNet)                                  --                        [32, 3, 244, 244]         [32, 3]                   Partial
├─Sequential (features)                                      --                        [32, 3, 244, 244]         [32, 1280, 8, 8]          False
│    └─Conv2dNormActivation (0)                              --                        [32, 3, 244, 244]         [32, 32, 122, 122]        False
│    │    └─Conv2d (0)                                       (864)                     [32, 3, 244, 244]         [32, 32, 122, 122]        False
│    │    └─BatchNorm2d (1)                                  (64)                      [32, 32, 122, 122]        [32, 32, 122, 122]        False
│    │    └─SiLU (2)                                         --                        [32, 32, 122, 122]        [32, 32, 12

## Create loss function and optimizer

In [13]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=0.01)

In [14]:
set_seeds(SEED=42)
engine.fit_fn(model=model,
              train_dataloader=train_dataloader,
              test_dataloader=test_dataloader,
              loss_fn=loss_fn,
              optimizer=optimizer,
              train_accuracy_fn=train_accuracy_fn,
              test_accuracy_fn=test_accuracy_fn,
              batch_size=BATCH_SIZE,
              epochs=5,
              device=device,
              experiment_name='auto_transform',
              model_name='effnetb0',
              extra='5_epochs')

[INFO] Created SummaryWriter, saving to: runs\22-02-2026\auto_transform\effnetb0\5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

samples remaning: 0 / 256
samples remaning: 64 / 256
samples remaning: 128 / 256
samples remaning: 192 / 256
Train loss: 0.8196098804473877 --- | Train Accuracy: 0.5733333230018616


 20%|██        | 1/5 [00:11<00:47, 11.83s/it]

Test loss: 0.5649888515472412 --- | Test Accuracy: 0.6933333277702332
samples remaning: 0 / 256
samples remaning: 64 / 256
samples remaning: 128 / 256
samples remaning: 192 / 256
Train loss: 0.5935109853744507 --- | Train Accuracy: 0.8222222328186035


...

Test loss: 0.40508347749710083 --- | Test Accuracy: 0.8533333539962769
samples remaning: 0 / 256
samples remaning: 64 / 256
samples remaning: 128 / 256
samples remaning: 192 / 256
Train loss: 0.410231351852417 --- | Train Accuracy: 0.9511111378669739


 80%|████████  | 4/5 [00:49<00:12, 12.37s/it]

Test loss: 0.3059160113334656 --- | Test Accuracy: 0.8666666746139526
samples remaning: 0 / 256
samples remaning: 64 / 256
samples remaning: 128 / 256
samples remaning: 192 / 256
Train loss: 0.6168246269226074 --- | Train Accuracy: 0.8933333158493042


100%|██████████| 5/5 [01:01<00:00, 12.30s/it]

Test loss: 0.48584458231925964 --- | Test Accuracy: 0.800000011920929
Total time taken on cuda is 1 min : 1.5201357000041753 secs
